# recount2 model building with CLAMP and PLIER

💡 **Environment:** `clamp-analyses`  

# CLAMP

## Load libraries

In [ ]:
# Create a timestamp to track the start of the analysis
start_time <- Sys.time()
cat("recount2 CLAMP and PLIER analysis started at:", format(start_time), "\n")

In [ ]:
if (!requireNamespace("PLIER", quietly = TRUE)) {
    devtools::install_github("wgmao/PLIER")
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(PLIER)
library(CLAMP)

source(here("config.R"))

set.seed(config$recount2$RANDOM_SVD_SEED)

## Output directory

In [90]:
output_data_dir <- config$recount2$DATASET_FOLDER
dir.create(output_data_dir, showWarnings = FALSE, recursive = TRUE)

## Settings

In [ ]:
block_size <- config$GENERAL$CHUNK_SIZE
N_CORES    <- config$recount2$N_CORES

## Download recount2 

In [91]:
url      <- config$recount2$URL
out_dir  <- here::here("data", "recount2")
zip_file <- file.path(out_dir, "recount2.zip")
rds_file <- file.path(out_dir, "recount2_PLIER_data", "recount_data_prep_PLIER.RDS")
rpkm_file <- file.path(out_dir, "recount2_PLIER_data", "recount_rpkm.RDS")

if (!file.exists(rds_file) && !file.exists(rpkm_file)) {
    if (!dir.exists(out_dir)) dir.create(out_dir, recursive = TRUE)
    
    system(paste('wget', url, '-O', zip_file))
    unzip(zip_file, exdir = out_dir)
} else {
    message("Required files already exist. Skipping download and decompression.")
}

Required files already exist. Skipping download and decompression.



## Preprocess recount2 data

In [92]:
# Read preprocessed recount2 data
data_mat <- readRDS(rds_file)

meta <- list()
meta$gene_symbols <- rownames(data_mat$rpkm.cm)
meta$samples <- colnames(data_mat$rpkm.cm)

rm(data_mat)

In [93]:
# Read in recount2 file

rpkm.df <- readRDS(rpkm_file)

mart <- biomaRt::useDataset("hsapiens_gene_ensembl",
                            biomaRt::useMart("ensembl"))

genes <- unlist(lapply(strsplit(rpkm.df$ENSG, "[.]"), `[[`, 1))

rpkm.df$ensembl_gene_id <- unlist(lapply(strsplit(rpkm.df$ENSG, "[.]"),
                                         `[[`, 1))
                                         
gene.df <- biomaRt::getBM(filters = "ensembl_gene_id",
                          attributes = c("ensembl_gene_id", "hgnc_symbol"),
                          values = genes,
                          mart = mart)

gene.df <- gene.df %>% dplyr::filter(complete.cases(.))

rpkm.df <- dplyr::inner_join(gene.df, rpkm.df,
                             by = "ensembl_gene_id")

rownames(rpkm.df) <- make.names(rpkm.df$hgnc_symbol, unique = TRUE)

rpkm.df <- rpkm.df %>% dplyr::select(-c(ensembl_gene_id:ENSG))

data_mat <- rpkm.df[meta$gene_symbols, meta$samples]

rm(rpkm.df)

data_mat <- as.matrix(as.data.table(data_mat))

Warning message in dplyr::inner_join(gene.df, rpkm.df, by = "ensembl_gene_id"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 23 of `x` matches multiple rows in `y`.
ℹ Row 29567 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


In [ ]:
fbm_files <- c(
  file.path(output_data_dir, "recount2.bk"),
  file.path(output_data_dir, "recount2.bk_preproc.bk"),
  file.path(output_data_dir, "recount2.bk_preproc_filtered.bk"),
  file.path(output_data_dir, "recount2.bk_preproc_filtered.rds")
)

for (f in fbm_files) {
  if (file.exists(f)) {
    res <- unlink(f)
    if (identical(res, 0L)) {
      message("✅ Removed existing FBM file: ", f)
    } else {
      warning("⚠️ Failed to remove: ", f)
    }
  } else {
    message("ℹ️ Not found, skipping: ", f)
  }
}

In [94]:
# Create the FBM
n_genes <- length(meta$gene_symbols)
n_samples <- length(meta$samples)

fbm_file <- file.path(output_data_dir, "FBMrecount2")
recount2FBM <- FBM(nrow = n_genes, ncol = n_samples, backingfile = fbm_file, create_bk = T)

In [ ]:
# Populate it with data
block_size <- config$GENERAL$CHUNK_SIZE
n_blocks <- ceiling(n_genes / block_size)

for (i in 1:n_blocks) {
  start_row <- (i-1) * block_size + 1
  end_row <- min(i * block_size, nrow(data_mat))
  recount2FBM[start_row:end_row, ] <- as.matrix(data_mat[start_row:end_row, ])
}

In [ ]:
# Preprocess and z‑score FBM
prep_recount2 <- preprocessCLAMPFBM(
  fbm        = recount2FBM,
  mean_cutoff= config$recount2$GENES_MEAN_CUTOFF,
  var_cutoff = config$recount2$GENES_VAR_CUTOFF
)

recount2_fbm_filt <- prep_recount2$fbm_filtered
recount2_rowStats <- prep_recount2$rowStats

Applying log2 transformation

Filling NAs with 0

Applying Z-score transformation



Check overlap genes with recount2 RNA-seq data and TWAS gene list

In [ ]:
twas_genes <- read.csv(here('output/creating_twas_gwas_list/gene_list_union_mashr_and_elastic_net_with_phi.csv'))
overlap_genes <- intersect(genes[prep_recount2$kept_rows], twas_genes$Gene_Symbol)

percent_overlap <- (length(overlap_genes) / length(twas_genes$Gene_Symbol)) * 100

cat("Number of overlapping genes:", length(overlap_genes), "\n")
cat("Percentage overlap (relative to TWAS genes):", round(percent_overlap, 2), "%\n")

In [ ]:
zscoreCLAMPFBM(recount2_fbm_filt, recount2_rowStats)

In [ ]:
zscoreCLAMPFBM(recount2_fbm_filt, recount2_rowStats, ncores=N_CORES)
recount2_genes <- genes[prep_recount2$kept_rows]

In [ ]:
saveRDS(samples, file = file.path(output_data_dir, "recount2_samples.rds"))
saveRDS(recount2_genes, file = file.path(output_data_dir, "recount2_genes.rds"))
saveRDS(recount2_fbm_filt, file = file.path(output_data_dir, "recount2_fbm_filt.rds"))

In [ ]:
df_recount2_fbm_filt <- as.data.frame(as.matrix(recount2_fbm_filt[]))
colnames(df_recount2_fbm_filt) <- samples
rownames(df_recount2_fbm_filt) <- recount2_genes
head(df_recount2_fbm_filt)

saveRDS(df_recount2_fbm_filt, file = file.path(output_data_dir, "df_recount2_fbm_filt.rds"))
write.csv(df_recount2_fbm_filt, file = file.path(output_data_dir, "df_recount2_fbm_filt.csv"))

## SVD computation

In [ ]:
if (!file.exists(file.path(output_data_dir, "recount2_svdRes.rds"))) {

  g_fb <- nrow(recount2_fbm_filt)
  samples_fb <- ncol(recount2_fbm_filt)
  SVD_K_recount2 <- min(g_fb, samples_fb) - 1

  message("Using SVD K = ", SVD_K_recount2)

  if (N_CORES > 1) {
    # disable BLAS parallelization when parallelizing
    options(bigstatsr.check.parallel.blas = FALSE)
    blas_nproc <- getOption("default.nproc.blas")
    options(default.nproc.blas = NULL)
  }

  recount2_svdRes <- big_randomSVD(
    recount2_fbm_filt,
    k = SVD_K_recount2,
    ncores = N_CORES
  )

  saveRDS(recount2_svdRes, file = file.path(output_data_dir, "recount2_svdRes.rds"))

  if (N_CORES > 1) {
    # restore previous state
    options(bigstatsr.check.parallel.blas = TRUE)
    options(default.nproc.blas = blas_nproc)
  }

} else {
  message("recount2_svdRes already exists, skipping SVD computation.")
}

## Estimate K for CLAMP

In [ ]:
recount2_fbm_filt <- readRDS(file.path(output_data_dir, "recount2_fbm_filt.rds"))
recount2_svdRes <- readRDS(file.path(output_data_dir, "recount2_svdRes.rds"))

CLAMP_K_recount2 <- num.pc(list(d = recount2_svdRes$d)) * 2
message("Inferred CLAMP K = ", CLAMP_K_recount2)

saveRDS(CLAMP_K_recount2, file = file.path(output_data_dir, "CLAMP_K_recount2.rds"))

Inferred PLIER K = 129



## CLAMPbase

In [ ]:
recount2_baseRes <- CLAMPbase(
  Y      = recount2_fbm_filt,
  svdres = recount2_svdRes,
  trace  = TRUE,
  clamp_k = CLAMP_K_recount2
)

****



[1] "L1 is set to 96.4556873352599"
[1] "L2 is set to 289.36706200578"
Progress 48 / 200 | Bdiff=0.000489, minCor=0.997171Converged at iteration= 48 | Bdiff=0.000489,  tol=0.000500     

In [ ]:
recount2_baseRes$Z <- data.frame(recount2_baseRes$Z)
rownames(recount2_baseRes$Z) <- recount2_genes
head(recount2_baseRes$Z)

recount2_baseRes$B <- data.frame(recount2_baseRes$B)
colnames(recount2_baseRes$B) <- samples
head(recount2_baseRes$B)

saveRDS(recount2_baseRes, file = file.path(output_data_dir, "CLAMPbase.rds"))

In [ ]:
model_dir <- file.path(output_data_dir, "CLAMPbase")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- recount2_baseRes$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- recount2_baseRes$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

## Prepare pathway priors

In [ ]:
recount2_gmtList <- list(
  KEGG = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=KEGG_2021_Human"),
  BP = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GO_Biological_Process_2025"),
  GTEx_Tissues = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GTEx_Tissues_V8_2023")
)

# prefix each gene‐set name with its library to guarantee uniqueness
for(lib in names(recount2_gmtList)) {
  names(recount2_gmtList[[lib]]) <- paste0(lib, "_", names(recount2_gmtList[[lib]]))
}

recount2_pathMat <- gmtListToSparseMat(recount2_gmtList)
recount2_matched <- getMatchedPathwayMat(recount2_pathMat, recount2_genes)
recount2_chatObj <- getChat(recount2_matched)

## CLAMPfull

In [ ]:
recount2_fullRes <- CLAMPfull(
    Y = recount2_fbm_filt,
    svdres = recount2_svdRes,
    priorMat = recount2_matched,
    clamp.base.result = recount2_baseRes,
    use_cpp = TRUE,
    trace = TRUE,
    clamp_k = CLAMP_K_recount2
  )

**PLIER v2 **



using provided PLIERbase result

L1=96.4556873352599; L2=289.36706200578



Converged at 108 / 350 | Bdiff=0.000011, minCor=0.999752olumns is 98, Number of annotated columns is 98, Number of annotated columns is 99, Number of annotated columns is 100, Number of annotated columns is 100, Number of annotated columns is 94, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 97, Number of annotated columns is 93, Number of annotated columns is 94, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 94, Number of annotated columns is 95, Number of annotated columns is 96, Number of annotated columns is 97, Number of annotated columns is 97, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 96, Number of annotated columns is 93, Number of annotated columns is 93, Number of annotate

Updating Z for CV

crossValidation

There are 68  LVs with AUC>0.70

There are 32  LVs with AUC>0.90



In [ ]:
recount2_baseRes$Z <- data.frame(recount2_baseRes$Z)
rownames(recount2_baseRes$Z) <- recount2_genes
head(recount2_baseRes$Z)

recount2_baseRes$B <- data.frame(recount2_baseRes$B)
colnames(recount2_baseRes$B) <- samples
head(recount2_baseRes$B)

recount2_fullRes$summary <- recount2_fullRes$summary %>%
    dplyr::rename(LV = LV_index)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

saveRDS(recount2_fullRes, file = file.path(output_data_dir, "CLAMPfull.rds"))

In [ ]:
model_dir <- file.path(output_data_dir, "CLAMPfull")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- recount2_fullRes$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- recount2_fullRes$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

summary <- recount2_fullRes$summary
write.csv(summary, file.path(model_dir, "summary.csv"))

In [ ]:
# Create a timestamp to track the end of the analysis
end_time <- Sys.time()
elapsed_time <- end_time - start_time
cat("Recount2 CLAMP analysis completed at:", format(end_time), "\n")
cat("Total elapsed time:", format(elapsed_time), "\n")

# PLIER

Run PLIER with the same inputs

In [ ]:
recount2_plier = PLIER::PLIER(
    recount2_fbm_filt[], 
    as.matrix(recount2_matched), 
    svdres = recount2_svdRes, 
    Chat = as.matrix(recount2_chatObj), 
    doCrossval = TRUE, 
    k = CLAMP_K_recount2
  )

Removing 0 pathways with too few genes



[1] 289.3671
[1] "L2 is set to 289.36706200578"
[1] "L1 is set to 144.68353100289"


errorY (SVD based:best possible) = 0.8092

New L3 is 0.00218749111818289

New L3 is 0.00193045413622771

New L3 is 0.00170361979580257

New L3 is 0.00193045413622771

New L3 is 0.00218749111818289

New L3 is 0.00218749111818289

New L3 is 0.00218749111818289

New L3 is 0.00218749111818289

New L3 is 0.00218749111818289

New L3 is 0.00193045413622771

New L3 is 0.00218749111818289

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

converged at  iteration 234 Bdiff is not decreasing

There are 53  LVs with AUC>0.70



In [ ]:
colnames(gtex_plier$Z) <- paste0('LV', seq_len(ncol(gtex_plier$Z)))

gtex_plier$summary <- gtex_plier$summary %>%
    dplyr::rename(LV = `LV index`)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

colnames(gtex_plier$B) <- samples

saveRDS(gtex_plier, file = file.path(output_data_dir, "PLIER.rds"))

LV 1,3.038562e-11,-1.164697e-13,1.499369e-11,2.420048e-11,1.076099e-11,2.400729e-11,2.450469e-11,2.529211e-11,2.492415e-11,2.411303e-11,⋯,-2.252487e-14,-9.868426e-13,7.459374e-13,-1.238694e-13,-5.471264e-13,3.208637e-13,-1.203242e-12,-9.133111e-13,-7.087071e-13,-7.032934e-13
"2,Cells - Cultured Fibroblasts Female 70-79 Up",-3.281172e-02,-3.819620e-02,-4.021169e-02,-4.102633e-02,-3.925897e-02,-3.957653e-02,-9.111154e-03,-7.944571e-03,-1.111526e-02,-5.022869e-03,⋯,3.445032e-02,9.405888e-02,5.244139e-02,9.127825e-02,8.897180e-02,6.534399e-02,7.464888e-02,1.037498e-01,4.380141e-02,9.150614e-02
"3,mRNA Splicing, via Spliceosome (GO:0000398)",-9.481710e-02,-7.740678e-02,-9.260198e-02,-8.277387e-02,-7.081893e-02,-8.502113e-02,-6.194781e-02,-6.563719e-02,-6.688297e-02,-7.041583e-02,⋯,-5.538873e-02,-6.185108e-02,-3.681471e-02,-3.960329e-02,-6.117308e-02,-2.970421e-02,-8.491269e-02,-3.287660e-02,-2.841848e-02,-1.169766e-01
"4,Translation (GO:0006412)",-1.167416e-01,-1.169561e-01,-1.241410e-01,-1.144214e-01,-1.081250e-01,-1.183666e-01,-1.348015e-01,-1.444428e-01,-1.385527e-01,-1.426251e-01,⋯,5.650564e-02,2.639246e-02,1.109185e-01,6.937041e-02,8.791064e-02,1.055854e-01,1.084018e-01,7.893540e-02,7.554681e-02,1.749875e-01
"5,Double-Strand Break Repair via Homologous Recombination (GO:0000724)",-1.475776e-02,-2.136897e-02,-2.183746e-02,-1.868801e-02,-1.880394e-02,-1.911732e-02,-2.733048e-02,-3.572729e-02,-3.150585e-02,-2.717683e-02,⋯,4.954903e-03,2.586807e-02,2.185875e-02,4.275850e-02,4.369622e-02,5.298591e-02,-1.144091e-02,5.217841e-02,-1.841314e-02,2.890525e-02
LV 6,1.137411e-02,1.100171e-02,2.021871e-02,2.901634e-02,1.232140e-02,1.205028e-02,9.152861e-03,1.060361e-02,-6.413729e-04,6.352040e-03,⋯,-7.302503e-02,-5.922457e-02,-7.198972e-02,-5.402172e-02,-6.231897e-02,-6.095302e-02,-5.806830e-02,-6.060223e-02,-7.126057e-02,-5.891329e-02


In [ ]:
model_dir <- file.path(output_data_dir, "PLIER")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- gtex_plier$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- gtex_plier$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

summary <- gtex_plier$summary
write.csv(summary, file.path(model_dir, "summary.csv"))